In [1]:
## importing libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier 
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
np.random.seed(1)

In [3]:
## Data loading

In [4]:
df=pd.read_csv("C:/Users/Jeshwant/Downloads/UniversalBank.csv")
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [5]:
## Dropping columns

In [6]:
df=df.drop(["ID","ZIP Code"],axis=1)

In [7]:
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


In [8]:
## Create dummies

In [9]:
df=pd.get_dummies(df, prefix=['Education'], columns=['Education'])
df.head()

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Education_1,Education_2,Education_3
0,25,1,49,4,1.6,0,0,1,0,0,0,1,0,0
1,45,19,34,3,1.5,0,0,1,0,0,0,1,0,0
2,39,15,11,1,1.0,0,0,0,0,0,0,1,0,0
3,35,9,100,1,2.7,0,0,0,0,0,0,0,1,0
4,35,8,45,4,1.0,0,0,0,0,0,1,0,1,0


In [10]:
## Split independent and dependent vars

In [11]:
df.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage',
       'Personal Loan', 'Securities Account', 'CD Account', 'Online',
       'CreditCard', 'Education_1', 'Education_2', 'Education_3'],
      dtype='object')

In [12]:
X = df[['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage',
       'Personal Loan', 'Securities Account', 'Online',
       'CreditCard', 'Education_1', 'Education_2', 'Education_3']]
y = df[['CD Account']]

In [13]:
## training and test sets split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [15]:

scaler = StandardScaler()

In [16]:
numbers =['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage',"Family"]
X_train[numbers] = scaler.fit_transform(X_train[numbers])
X_test[numbers] = scaler.transform(X_test[numbers])

In [17]:
## model the data

In [18]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

In [19]:
## logistic regression

In [20]:
log_reg_model = LogisticRegression(penalty='none')
_ = log_reg_model.fit(X_train, np.ravel(y_train))

In [21]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978,1.0,0.60241,0.75188


In [22]:
## RandomSearchCV Logistic Regression

In [23]:
score_measure = "recall"
LR=LogisticRegression()
kfolds = 5
param_grid = {'C': [0.1, 1, 10,0.001], 
              "solver" : [ 'lbfgs', 'liblinear'],
              "penalty" : ['l1','l2','lasso','elastic']} 
  
grid = RandomizedSearchCV(LR, param_grid, refit = True, verbose = 3)
  
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .C=1, penalty=l2, solver=liblinear;, score=0.974 total time=   0.0s
[CV 2/5] END .C=1, penalty=l2, solver=liblinear;, score=0.981 total time=   0.0s
[CV 3/5] END .C=1, penalty=l2, solver=liblinear;, score=0.980 total time=   0.0s
[CV 4/5] END .C=1, penalty=l2, solver=liblinear;, score=0.979 total time=   0.0s
[CV 5/5] END .C=1, penalty=l2, solver=liblinear;, score=0.983 total time=   0.0s
[CV 1/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.961 total time=   0.0s
[CV 2/5] END ...C=0.1, penal

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

[CV 3/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.980 total time=   0.0s
[CV 4/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.977 total time=   0.0s
[CV 5/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.983 total time=   0.0s
[CV 1/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=10, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=10, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=10, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=10, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=1

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

[CV 1/5] END .C=1, penalty=l1, solver=liblinear;, score=0.974 total time=   0.0s
[CV 2/5] END .C=1, penalty=l1, solver=liblinear;, score=0.981 total time=   0.0s
[CV 3/5] END .C=1, penalty=l1, solver=liblinear;, score=0.980 total time=   0.0s
[CV 4/5] END .C=1, penalty=l1, solver=liblinear;, score=0.977 total time=   0.0s
[CV 5/5] END .C=1, penalty=l1, solver=liblinear;, score=0.983 total time=   0.0s
The best recall score is 0.9794285714285715
... with parameters: {'solver': 'liblinear', 'penalty': 'l2', 'C': 1}


C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.97942857        nan 0.96685714 0.97914286        nan        nan
        nan        nan 0.97914286 0.97914286]
  warnings.warn(
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [24]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Randomised", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [25]:
## Gridsearchcv logistic regression

In [26]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10], 
              'solver' : [ 'lbfgs', 'liblinear'],
              'penalty' : ['l1','l2','lasso','elastic']} 
  
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
  
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.974 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.981 total time=   0.0s
[CV 3/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.980 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.977 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.983 total time=   0.0s
[CV 1/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.961 total time=   0.0s
[CV 2/5] END ...C=0.1, penalty=l2, solver=

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

[CV 5/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

...

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END C=10, penalty=l1, solver=liblinear;, score=0.983 total time=   0.1s
[CV 1/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.974 total time=   0.0s
[CV 2/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.981 total time=   0.0s
[CV 3/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.980 total time=   0.0s
[CV 4/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.977 total time=   0.0s
[CV 5/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.983 total time=   0.0s
[CV 1/5] END C=10, penalty=l2, solver=liblinear;, score=0.974 total time=   0.0s
[CV 2/5] END C=10, penalty=l2, solver=liblinear;, score=0.981 total time=   0.0s
[CV 3/5] END C=10, penalty=l2, solver=liblinear;, score=0.980 total time=   0.0s
[CV 4/5] END C=10, penalty=l2, solver=liblinear;, score=0.977 total time=   0.0s
[CV 5/5] END C=10, penalty=l2, solver=liblinear;, score=0.983 total time=   0.0s
[CV 1/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ...C=10, penalt

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

[CV 3/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .C=10, pen

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
75 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise Val

In [27]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Grid", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [28]:
## SVM Linear kernel

In [29]:
svm_lin_model = SVC(kernel="linear", probability=True)
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [30]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with linear kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978,1.0,0.60241,0.75188
0,Logistic Regression Randomised,0.978,1.0,0.60241,0.75188
0,Logistic Regression Grid,0.978,1.0,0.60241,0.75188
0,svm with linear kernel,0.978,1.0,0.60241,0.75188


In [31]:
## SVM rbf kernel

In [32]:
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale', probability=True)
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [33]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with rbf kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.00000,0.602410,0.751880
0,Logistic Regression Randomised,0.978000,1.00000,0.602410,0.751880
0,Logistic Regression Grid,0.978000,1.00000,0.602410,0.751880
0,svm with linear kernel,0.978000,1.00000,0.602410,0.751880
0,svm with rbf kernel,0.973333,0.90566,0.578313,0.705882


In [34]:
## SVM poly kernel

In [35]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10, probability=True)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [36]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)

In [37]:
## Randomsearchcv svm

In [38]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10], 
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['linear','poly','rbf']} 
  
rand_search = RandomizedSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.974 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.981 total time=   0.0s
[CV 3/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.980 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.977 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.983 total time=   0.0s
[CV 1/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.961 total time=   0.0s
[CV 2/5] END ...C=0.1, penalty=l2, solver=

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

[CV 1/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END .C=1, penalty=l1, solver=liblinear;, score=0.974 total time=   0.0s
[CV 2/5] END .C=1, penalty=l1, solver=liblinear;, score=0.981 total time=   0.0s
[CV 3/5]

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

[CV 2/5] END .....C=1, penalty=l2, solver=lbfgs;, score=0.981 total time=   0.0s
[CV 3/5] END .....C=1, penalty=l2, solver=lbfgs;, score=0.980 total time=   0.0s
[CV 4/5] END .....C=1, penalty=l2, solver=lbfgs;, score=0.977 total time=   0.0s
[CV 5/5] END .....C=1, penalty=l2, solver=lbfgs;, score=0.983 total time=   0.0s
[CV 1/5] END .C=1, penalty=l2, solver=liblinear;, score=0.974 total time=   0.0s
[CV 2/5] END .C=1, penalty=l2, solver=liblinear;, score=0.981 total time=   0.0s
[CV 3/5] END .C=1, penalty=l2, solver=liblinear;, score=0.980 total time=   0.0s
[CV 4/5] END .C=1, penalty=l2, solver=liblinear;, score=0.979 total time=   0.0s
[CV 5/5] END .C=1, penalty=l2, solver=liblinear;, score=0.983 total time=   0.0s
[CV 1/5] END ....C=1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ....C=1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ....C=1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ....C=1, penalt

...

[CV 5/5] END C=10, penalty=l1, solver=liblinear;, score=0.983 total time=   0.1s
[CV 1/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.974 total time=   0.0s
[CV 2/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.981 total time=   0.0s
[CV 3/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.980 total time=   0.0s
[CV 4/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.977 total time=   0.0s
[CV 5/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.983 total time=   0.0s
[CV 1/5] END C=10, penalty=l2, solver=liblinear;, score=0.974 total time=   0.0s
[CV 2/5] END C=10, penalty=l2, solver=liblinear;, score=0.981 total time=   0.0s
[CV 3/5] END C=10, penalty=l2, solver=liblinear;, score=0.980 total time=   0.0s
[CV 4/5] END C=10, penalty=l2, solver=liblinear;, score=0.977 total time=   0.0s
[CV 5/5] END C=10, penalty=l2, solver=liblinear;, score=0.983 total time=   0.0s
[CV 1/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ...C=10, penalt

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

[CV 5/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=10, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=10, p

C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
75 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Jeshwant\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise Val

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_score_'

In [41]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random search SVM Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

NotFittedError: This RandomizedSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [40]:
## Gridsearchcv svm

In [ ]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

In [ ]:
c_matrix = confusion_matrix(y_test, grid.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search SVM Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [ ]:
## Decisiontree

In [ ]:
classifier = DecisionTreeClassifier()
classifier = classifier.fit(X_train,y_train)

In [ ]:
c_matrix = confusion_matrix(y_test, classifier.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"DT", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [ ]:
## Decisontree randomsearchcv

In [ ]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,60),  
    'min_samples_leaf': np.arange(1,50),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1, return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

In [ ]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision tree random search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [ ]:
## Decisiontree gridsearchcv

In [ ]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(25,32),  
    'min_samples_leaf': np.arange(3,6),
    'min_impurity_decrease': np.arange(0.0001, 0.0004, 0.0001),
    'max_leaf_nodes': np.arange(194,200), 
    'max_depth': np.arange(15,21), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

In [ ]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search DT", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [ ]:
performance.sort_values(by =['Recall'])